# Change working directory

In [ ]:
%cd /kaggle/working
%ls

# Libraries
- cv2: load image and draw rectangle (imread,  cvtColor, rectangle)
- random: shuffle the list (shuffle)
- pandas: create DataFrame
- matplotlib: show images and plot graph
- os: work with file

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os, cv2, random
import pandas as pd

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Convolution2D,MaxPooling2D, Dense, Flatten, InputLayer
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.backend import clear_session
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Unzip files

In [ ]:
import zipfile
input_path = '/kaggle/input/dogs-vs-cats'
work_path = '/kaggle/working/data'
train_path = os.path.join(input_path,'train.zip')
test_path = os.path.join(input_path,'test1.zip')

with zipfile.ZipFile(train_path, 'r') as zip_ref:
    zip_ref.extractall(work_path)
with zipfile.ZipFile(test_path, 'r') as zip_ref:
    zip_ref.extractall(work_path)

# Create directory path

In [ ]:
data_dir = work_path
train_dir = data_dir+"/train"
test_dir = data_dir+"/test1"

# Create dataframe and data generator
- Frame structure: { 'filename': [...], 'class': ['dog',...] }
- Create the frame with the struct above and shuffle random (frac=1)
- We do the data augmentation and shuffle with train data only (shuffle validation set does not achive anything)
- Splitting 80% for train, 20% for testing
- Each set, we divide into multiple batchs
- Because of having only 2 classes, the mode will be binary

In [ ]:
df = pd.DataFrame()
fnames = os.listdir(train_dir)

class_name = []
for name in fnames:
    class_name.append(name.split('.')[0])
    
data = {'filename':fnames,'class':class_name}
df = pd.DataFrame(data)
df = df.sample(frac=1)

train_datagen = ImageDataGenerator(rescale = 1/255,
                                   rotation_range=20,
                                   shear_range=0.2,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True,
                                   zoom_range=0.2)
valid_datagen = ImageDataGenerator(rescale = 1/255)

idx = int(0.8*len(df))
train_df = df.iloc[:idx]
valid_df = df.iloc[idx:]

target = (224,224)

train_set = train_datagen.flow_from_dataframe(train_df,
                                              directory=train_dir,
                                              shuffle=True,
                                              target_size = target,
                                              batch_size = 64,
                                              class_mode = 'binary')

valid_set = valid_datagen.flow_from_dataframe(valid_df,
                                              directory=train_dir,
                                              shuffle=False,
                                              target_size = target,
                                              batch_size = 32,
                                              class_mode = 'binary')

# VGG 1-block model
- This is the model with VGG-style using 1 block of Conv2D and Pooling
- Using the learning rate 0.05
- Momentum .9 and Nesterov (NAG optimizer)
- Loss function is binary cross entropy (binary classification cat or dog only)

In [ ]:
clear_session()

model = Sequential([
    InputLayer(input_shape=target+(3,)),
    Convolution2D(32,3,activation='relu'),
    MaxPooling2D(2),
    Flatten(),
    Dense(512,activation='relu'),
    Dense(1,activation='sigmoid')
])
model.summary()

opt = SGD(learning_rate = 0.05,
          momentum = 0.9,
          nesterov = True)
model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['acc'])
model.optimizer.get_config()

# VGG 2-block model
- This is the model with VGG-style using 2 block of Conv2D and Pooling
- Using the learning rate 0.05
- Momentum 0.9 and Nesterov (NAG optimizer)
- Loss function is binary cross entropy (binary classification cat or dog only)

In [ ]:
clear_session()

model = Sequential([
    InputLayer(input_shape=target+(3,)),
    Convolution2D(32,3,activation='relu'),
    MaxPooling2D(2),
    Convolution2D(64,3,activation='relu'),
    MaxPooling2D(2),
    Flatten(),
    Dense(512,activation='relu'),
    Dense(1,activation='sigmoid')
])
model.summary()

opt = SGD(learning_rate = 0.05,
          momentum = 0.9,
          nesterov = True)
model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['acc'])
model.optimizer.get_config()

# VGG 3-block model
- This is the model with VGG-style using 3 block of Conv2D and Pooling
- Using the learning rate 0.05
- Momentum 0.9 and Nesterov (NAG optimizer)
- Loss function is binary cross entropy (binary classification cat or dog only)


In [ ]:
clear_session()

model = Sequential([
    InputLayer(input_shape=target+(3,)),
    Convolution2D(32,3,activation='relu'),
    MaxPooling2D(2),
    Convolution2D(64,3,activation='relu'),
    MaxPooling2D(2),
    Convolution2D(128,3,activation='relu'),
    MaxPooling2D(2),
    Flatten(),
    Dense(512,activation='relu'),
    Dense(1,activation='sigmoid')
])
model.summary()

opt = SGD(learning_rate = 0.05,
          momentum = 0.9,
          nesterov = True)
model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['acc'])
model.optimizer.get_config()

# VGG-16 modified model
- Using pre-trained model VGG-16 without top layers
- The VGG-16 required image of shape (224,224)
- We freeze the trained parameters of the model
- Then we add our top layer by our self

In [ ]:
clear_session()

model = VGG16(include_top=False, input_shape=target+(3,))
for layer in model.layers:
    layer.trainable = False
flat1 = Flatten()(model.layers[-1].output)
class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
output = Dense(1, activation='sigmoid')(class1)
model = Model(inputs=model.inputs, outputs=output)

model.summary()

opt = SGD(learning_rate = 0.001,
          momentum = 0.9)
model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['acc'])
model.optimizer.get_config()

# Train model
In this section, we train the model with the checkpoint and early stopping method:

- Checkpoint:
    - We save only the best model which have highest "val acc" value
    - We save whole the model not the weight only
    - Mode is auto which means we choose model whose the maximum value "val acc"
- Early stopping:
    - We also use the "val acc" value of the model to evaluate
    - If the model is not improve after 10 epochs, we will stop trainning (it may be converged)
- Trainning:
    - The iteration of each epoch can be calculated by: number of data / batch size (we use all the batches to train)

In [ ]:
checkpoint = ModelCheckpoint("temp_model.h5", 
                             monitor='val_acc', 
                             verbose=1, 
                             save_best_only=True, 
                             save_weight_only=False, 
                             mode='auto')

early = EarlyStopping(monitor='val_acc', 
                      min_delta=0, 
                      patience=10, 
                      verbose=1, 
                      mode='auto')

history = model.fit(train_set,
                    validation_data=valid_set,
                    steps_per_epoch=train_set.n//train_set.batch_size,
                    validation_steps=valid_set.n//valid_set.batch_size,
                    epochs = 50,
                    callbacks=[checkpoint,early])

# Plot the loss and accuracy of train and validation
- This step can help you detect the overfitting problem by looking at learning curve

In [ ]:
legend = ['train','validation']

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title("Accuracy")
plt.xlabel("epochs")
plt.ylabel("acc")
plt.legend(legend,loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("Binary cross-entropy loss")
plt.xlabel("epochs")
plt.ylabel("loss")
plt.legend(legend,loc='upper left')
plt.show()

# Save the model down

In [ ]:
model.save('my_model.h5')

# Load the model up

In [ ]:
model = load_model('my_model.h5')
model.summary()

# Test the accuracy after load the model

In [ ]:
print("Accuracy:",model.evaluate(valid_set)) # (loss, accuracy)

# Extract all the conv layer to visualize the prediction
- We take all the output of layer whose name contains 'conv' 
- Then we create a new model whose outputs are those layers
- This step is to understand what is working inside the model

In [ ]:
layer_outputs = []
for layer in model.layers:
    if 'conv' not in(layer.name):
        continue
    layer_outputs.append(layer.output)

activation_model = Model(inputs = model.input, outputs = layer_outputs)

# Define functions
- Preprocess: Use to resize the function to the target size and map the value of pixel to [0,1]
- Predict: Create the batch and get the proabiblity of the model
- Get label: Get the label of the prediction from the probability
- Visualize: Visualize the conv layers output
- Get box: Get the best predicted box which contains the dog/cat using sliding window

In [ ]:
def preprocess(img):
    img = cv2.resize(img,target)
    img = img/255
    return np.array(img)
    
def predict(img):
    img = preprocess(img)
    img = img.reshape((1,)+img.shape)
    probability = model.predict(img)
    return probability

def getLabel(probability):
    if probability<0.5:
        probability=0
    else:
        probability=1
    return list(train_set.class_indices)[probability]

def visualize(img):
    img = preprocess(img)
    img = img.reshape((1,)+img.shape)
    fmaps = activation_model.predict(img)

    for i in range(len(fmaps)):
        activation = fmaps[i]
        
        fig = plt.figure(figsize=(20,15))
        fig.suptitle(layer_outputs[i].name)
        
        for j in range(min(8*8,activation.shape[-1])):
            plt.subplot(8,8,j+1)
            plt.imshow(activation[0,:,:,j],cmap='gray')
    plt.show()

WIN_SIZES=[]
for i in range(100,260,20):
    WIN_SIZES.append(i)
    
def get_box(img,step=20,win_sizes=WIN_SIZES):
    best_box = None
    best_distance = 1
    raw_prob = predict(img)
    if (raw_prob<0.5):
        raw_prob=0
    else:
        raw_prob=1
        
    for win in win_sizes:
        print("Run with window size:",str(win))
        for top in range(0,img.shape[0]-win+1,step):
            for left in range(0,img.shape[1]-win+1,step):
                box = (left,top,left+win,top+win)
                crop = img[box[1]:box[3],box[0]:box[2]]
                prob = predict(crop)
                distance = abs(raw_prob-prob)
                if (distance<best_distance):
                    best_box = box
                    best_distance = distance
    return (best_box, best_distance)

# Testing predict with random images

In [ ]:
test_fnames = os.listdir(test_dir)
random.shuffle(test_fnames)
result = []

nRow = 5
nCol = 5
nPic = nRow*nCol

fig, ax = plt.subplots(nRow,nCol,figsize=(30,20))

for i, fnames in enumerate(test_fnames):
    if (i==nPic):
        break
    
    pred_path = os.path.join(test_dir,fnames)
    img = cv2.imread(pred_path)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    
    col = i%nCol
    row = i//nCol
    title = getLabel(predict(img))
    ax[row][col].imshow(img)
    ax[row][col].set_title(title)
plt.show()    

# Testing predict one image

In [ ]:
# Load the image
test_fnames = os.listdir(test_dir)
idPic = random.randint(0,len(test_fnames))
pred_path = os.path.join(test_dir,test_fnames[idPic])
img = cv2.imread(pred_path)
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

# Get the prediction
title = getLabel(predict(img))
        
# Plot the image and result
plt.imshow(img)
plt.title(title)
plt.show()

# Visualize CNN output's layers

In [ ]:
# Visualize the conv layers
visualize(img)

# Running sliding window

In [ ]:
# Finding the best fit box for detection
box, best_distance = get_box(img)
print("Probability:",float(1-best_distance))
startPoint = (box[0],box[1])
endPoint = (box[2],box[3])

# Draw rectangle and show image
COLOR=(255,0,0)
img = cv2.rectangle(img,startPoint,endPoint,COLOR,2)
plt.imshow(img)

# Output for submission

In [ ]:
submission = pd.DataFrame()

test_fnames = os.listdir(test_dir)
result = []

pre_percent = 0
for i,fnames in enumerate(test_fnames):
    pred_path = os.path.join(test_dir,fnames)
    img = cv2.imread(pred_path)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    result.append(float(np.round(np.reshape(predict(img),-1))))
    
    percent = i/len(test_fnames)
    if (pre_percent+0.1<=percent):
        pre_percent+=0.1
        print("Done:",str(percent*100)+'%')

for i in range(len(test_fnames)):
    test_fnames[i] = test_fnames[i].split('.')[0]
submission['id'] = test_fnames
submission['label'] = result
submission.to_csv('submission.csv',index=False)